In [4]:
import pandas as pd
import numpy as np
import importlib

import archs4py as a4

importlib.reload(a4)
a4.reload()

# Load existing samples

Query sample queue to extract previously aligned samples. These samples can be used to filter the total set of samples found in the specified platforms. The remaining samples are new samples that can be added to the queue.

In [7]:
processed_gsm = a4.db.processed_gsms()
srr = a4.sra.get_gsms()

c:\Users\Rubberchicken\Documents\GitHub\archs4py\archs4py\sra.py:26: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  return read()


# Load SRAdb

Load all SRAdb info and extract all new SRA samples that are not specified as previously aligned in `srr`.

In [10]:
new_samples = a4.geo.scan_platforms(srr, processed_gsm)

100%|██████████| 22/22 [44:51<00:00, 122.32s/it]


In [3]:
new_samples.shape[0]

NameError: name 'new_samples' is not defined

## Add samples to queue

If new samples exist, iterate over the new samples and add them to the database. After adding to the queue manual instance scaling in AWS has to be performed. The instances will then automatically be processed.

In [ ]:
if new_samples.shape[0] > 0:
    maxlistid = a4.db.get_max_id()
    gsm_meta = new_samples
    gsm_meta["listid"] = list(range(maxlistid+1, maxlistid+gsm_meta.shape[0]+1))
    sample_mapping = gsm_meta.loc[:, ["gse", "gsm", "sra", "listid"]]
    sample_mapping = sample_mapping.sort_values("gse").reset_index(drop=True)
    a4.db.add_mapping(sample_mapping)
    a4.db.add_sequencing_jobs(new_samples)

,gse,gsm,sra,listid
0,GSE100064,GSM2670741,SRR5687234,1583729
1,GSE100064,GSM2670740,SRR5687233,1583728
2,GSE100391,GSM2680090,SRR5750314,1583730
3,GSE100391,GSM2680182,SRR5750406,1647514
4,GSE100391,GSM2680181,SRR5750405,1647513
...,...,...,...,...
290052,GSE98548,GSM2598508,SRR5508144,1556867
290053,GSE98548,GSM2598507,SRR5508143,1556866
290054,GSE98548,GSM2598506,SRR5508142,1556865
290055,GSE98548,GSM2598505,SRR5508141,1556864


## Post processing of alignment data

This step is performed after the queue has been successfully aligned. There will be some samples that failed alignment. First we check that there are no more waiting samples in the queue before processing with building.

It is recommended to run post processing on an AWS instance. There can be significant data transfer from S3 which can incurre cost for data egress. By keeping processing in AWS this cost will not be incurred.

There are some system requirements to consider. Sufficient memory will be needed to build some of the 


In [5]:
a4.db.waiting_jobs()

,0
0,0


In [1]:
import os

organism = "mouse"  # can also be 'mouse'

author = "Alexander Lachmann"
lab = "Ma'ayan Lab - Icahn School of Medicine at Mount Sinai"
contact = "alexander.lachmann@mssm.edu"
version = 9

os.makedirs('downloads', exist_ok=True)
os.makedirs('downloads/counts', exist_ok=True)

In [ ]:
def split_list(alist, parts=1):
    length = len(alist)
    return [ alist[i*length // parts: (i+1)*length // parts] for i in range(parts) ]

In [ ]:
os.system("cp downloads/"+organism+'_matrix.h5', 'downloads/'+organism+'_matrix_2.h5')